In [ ]:
import os
import sys
import logging

import numpy as np

from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

import lsst.observing.commands.calibrationStarVisit as calibrationStarVisit

Make sure that logs aren't lost

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.ERROR

In [ ]:
if True:
    attcs = ATTCS()
    latiss = LATISS(attcs.domain)

Execute this block if you want to pass `display=display` to `calibrationStarVisit.takeData`

In [ ]:
if False:
    import lsst.afw.display as afwDisplay

    afwDisplay.setDefaultBackend("matplotlib" if False else "firefly")
    os.environ['FIREFLY_URL'] = "http://firefly01.cp.lsst.org:8080/firefly/"

    disp = afwDisplay.Display(1, name='RHL', reopenPlot=True)

    if afwDisplay.getDefaultBackend() == "firefly":
        disp.getClient().display_url()
else:
    disp = None

In [ ]:
for repo in ["/mnt/dmcs/oods_butler_repo/repo",
             "/project/shared/auxTel",
             os.path.expanduser("~/Data/auxTel" "XX"),
            ]:
    if os.path.exists(repo):
        break

print(f"Using {repo}")  
repo = os.path.join(repo, "rerun", "quickLook")

if os.path.exists(repo):
    from lsst.daf.persistence import Butler
else:
    print(f"I can't find {repo}; using butler emulator")
    from lsst.ts.standardscripts.auxtel.butler import Butler
    
    latiss.domain.time_per_second = 0.0

butler = Butler(repo)
dataId = dict(dayObs="2020-02-21")

In [ ]:
repo = "/mnt/dmcs/oods_butler_repo/repo/rerun/quickLook"

if os.path.exists(repo):
    from lsst.daf.persistence import Butler
else:
    print(f"I can't find {repo}; using emulator")
    from lsst.ts.standardscripts.auxtel.butler import Butler
    
    latiss.domain.time_per_second = 0.0

butler = Butler(repo)
dataId = dict(dayObs="2020-02-21")

obslog = logging.getLogger()
obslog.setLevel(logging.INFO)

N.b. If `doPointingModel` and we don't have a pointing model, exec the next cell

In [ ]:
if False:
    await attcs.atptg.cmd_pointNewFile.start()

Take a standard set of exposures of a star

In [ ]:
if True:
    objectName = "HD 185975"    # pole
else:
    objectName = "HD 146233"

expTime0 = await calibrationStarVisit.takeData(attcs, latiss, butler,
                                               objectName, doPointingModel=True, logger=obslog, silent=True)

Define our own set of exposures and take them

In [ ]:
exposures = [
    ('empty_1',    45, 'ronchi90lpmm'),
    ('quadnotch1', 45,           None),
]

await calibrationStarVisit.takeData(attcs, latiss, butler,
                                    objectName, exposures, updateFocus=False,
                                    logger=obslog, display=disp)